In [ ]:
import sqlite3
import pandas

conn = sqlite3.connect('proteins.db')

cursor = conn.cursor()

In [22]:
def print_protein_info(df):
    """Prints the protein information from the DataFrame."""
    if df.empty:
        print("No data found.")
        return
    
    for index, row in df.iterrows():
        print(f"Name: {row['name']}")
        print(f"Accession: {row['accession']}")
        print(f"Function: {row['function']}")
        print(f"Domains: {row['domains']}")
        print(f"CRISPR Score: {row['crispr_score']}")
        print(f"References: {row['reference']}")
        print("--------------------------------------------------\n")

In [31]:
# query = "SELECT * FROM proteins WHERE accession = 'TGME49_222170';"
query = "SELECT * FROM proteins WHERE name LIKE '%GRA%' AND crispr_score <=0;"
df = pandas.read_sql_query(query, conn)
df.sort_values(by="name", inplace=True)
df

,id,name,accession,function,domains,crispr_score,reference
0,13,GRA01,TGME49_270250,Potential function in suppression of host apop...,"signal peptide, calcium binding domain, no tra...",-5.27,(Cesbron-Delauw et al. 1989; Wu et al. 2017)
1,22,GRA10,TGME49_268900,Required for parasite growth. Located in the P...,"transmembrane domain, no signal peptide",-4.45,(Nam 2009; Mercier and Cesbron-Delauw 2015; Wi...
2,27,GRA17,TGME49_222170,Required for PVM permeability and host nutrien...,no signal peptide or transmembrane domain,-2.93,"(Gold et al. 2015); (Paredes-Santos et al., 2019)"
3,29,GRA19,TgME49_087740,Interacts with and may have a role at the PVM,no signal peptide or transmembrane domain,-1.66,(Hsiao et al. 2013; Mercier and Cesbron-Delauw...
4,43,GRA36,213067_TGGT1,Not essential in RH strain,"signal peptide, transmembrane domains",-0.21,(Bai et al. 2018)
5,45,GRA38,312420_TGGT1,hypothetical protein possibly located in the PV,"signal peptide, no transmembrane domain",-1.15,(Bai et al. 2018; Nadipuram et al. 2016)
6,46,GRA39,TGME49_289380,virulence factor that can interfere with host ...,signal peptide no transmembrane domain,-4.80,(Nadipuram et al. 2016)
7,51,GRA44,TGME49_228170,"Putative phosphatase, acid-phosphatase associa...","domain, no signal peptide, transmembrane domain",-3.28,(Panas et al. 2019; Blakely et al. 2020)
8,54,GRA47,TGME49_254000,hypothetical protein important for parasite su...,no signal peptide or transmembrane domain,-3.49,(Beraki et al. 2019; Tu et al. 2019)
9,59,GRA52/CST5,TGME49_319340,cyst wall proteins,"signal peptide, transmembrane domains",-3.96,(Tu et al. 2019)
